# markdown

In [4]:
def dequote(s: str):
    """
    If a string has single or double quotes around it, remove them.
    Make sure the pair of quotes match.
    If a matching pair of quotes is not found, return the string unchanged.
    """
    if (s[0] == s[-1]) and s.startswith(("'", '"')):
        return s[1:-1]
    return s

In [5]:
import pandas as pd
import requests

def get_track_downloads(show_name: str, client_id: str) -> pd.DataFrame:
    url = 'http://api.soundcloud.com/resolve?url=http://soundcloud.com/' + show_name + '/tracks&client_id=' + client_id
    tracks = requests.get(url=url)
    tracks = tracks.json()

    stats = []
    for track in tracks:
        url2 = "https://api-v2.soundcloud.com/tracks/" + str(track['id']) + "?client_id=" + client_id
        res = requests.get(url=url2).json()
        stats.append([res['title'], res['playback_count']])

    stats = pd.DataFrame(stats, columns=['title', 'playback_count'])
    stats['title'] = [dequote(title) for title in stats.title]
    stats = stats.set_index('title')
    return stats


In [6]:
from datetime import timedelta, datetime

client_id = 'DjQCXgNUkh7MZaqgtwPYHqeyOwdKFdPd'
show_name = "blockchannelshow"

tracks = get_track_downloads(show_name=show_name, client_id=client_id)

date = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')

In [7]:
date

'2017-10-01'

In [8]:
tracks = tracks.reset_index()

In [9]:
tracks

,title,playback_count
0,Episode 30: To Bee or Not to Bee,130
1,Episode 29: Making Sense of Cent.co,204
2,"Episode 28: Securing the Metaverse, with Decen...",263
3,Episode 27: Po.et and We Didn't Even Know It,196
4,"Episode 26: Off the Grid, with Grid+",428
5,"Episode 25: Building Digital Communities, with...",337
6,"Episode 24: Bringing Crypto to the World, with...",364
7,"Episode 23: Unmasking the Future, with Metamask",367
8,"Episode 22: Tackling the Real Estate Dinosaur,...",529
9,Episode 21: Building Unstoppable Organizations...,456


In [10]:
def create_json(title, count) -> dict:
    return {
        "measurement": "blockchannel",
        "tags": {
            "episode": title,
        },
        "fields": {
            "value": count
        }
    }
    

In [11]:
tracks['json'] = [create_json(title, count) for title,count in zip(tracks.title.values, tracks.playback_count.values)]

In [12]:
import influxdb

In [23]:
client = influxdb.InfluxDBClient(host="192.168.1.249", database='sc_downloads')

In [24]:
array = []
for json_object in tracks.json.values:
    array.append(json_object)
client.write_points(array)

True

In [25]:
client.query(database='sc_downloads', query='SELECT "value" FROM "blockchannel" WHERE "episode"=\''+tracks.title.values[2]+'\';')

ResultSet({'('blockchannel', None)': [{'time': '2017-10-02T07:23:32.963863138Z', 'value': 263}]})

In [27]:
client.query(database='sc_downloads', query='SELECT "value" FROM "blockchannel"')

ResultSet({'('blockchannel', None)': [{'time': '2017-10-02T07:23:32.963863138Z', 'value': 248}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 364}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 1699}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 649}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 911}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 906}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 884}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 1063}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 130}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 1954}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 204}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 263}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 196}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 428}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 337}, {'time': '2017-10-02T07:23:32.963863138Z', 'value': 364}, {'time': '2017-10-02T07:23:32.9

In [22]:
tracks.title.values[2]

'Episode 28: Securing the Metaverse, with Decentraland'

In [16]:
client.create_database('sc_downloads')

In [20]:
client.create_retention_policy(name='infinite', duration="INF", replication='1' , default=True)